The goal of this notebook is to create a geodaframe containing all the necessary information about each solar panel annotation. This dataframe will be useful in creating image mask pairs of tiles (896x896 pixels) of the Cape Town geotiff images. The code to create the geodataframe and the image make pairs is available in this notebook.   

In [ ]:
# import necessary packages
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import rasterio
import ast
import cv2
import re
import shutil
import random
from rasterio.transform import rowcol
from rasterio.windows import Window
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import box
from collections import namedtuple
from tqdm import tqdm
from data_generation import DatasetGenerator
from PIL import Image

Determine the images that have been annotated

In [30]:
status = pd.read_excel(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\CapeTown_ImageIDs.xlsx")
completed = status[status["Status"]=='Completed']
completed = completed[['Image ID', 'Annotator']]
completed

c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Image ID,Annotator
56,W07A_1,Gary Alvarez Mejia
63,W07A_16,Abby Finkle
72,W07A_24,Vanshika Mittal
106,W07C_10,Biz Yoder
107,W07C_11,Biz Yoder
...,...,...
2541,W57B_6,Fiona Bolte-Bradhurst
2542,W57B_7,Fiona Bolte-Bradhurst
2543,W57B_8,Fiona Bolte-Bradhurst
2544,W57B_9,Fiona Bolte-Bradhurst


There are 2807 images of Cape Town in total. As of Nov 29, there 544 completed images, including those that have no solar panel annotation. 151 geotiff images have annotations.

In [31]:
completed['Annotator'].unique()

array(['Gary Alvarez Mejia', 'Abby Finkle', 'Vanshika Mittal',
       'Biz Yoder', 'Zeinab Mukhtar', 'Ye Khaung Oo', 'Brian Mulu Mutua',
       'Halle Evans', 'Ummamah Shah', 'Fiona Bolte-Bradhurst'],
      dtype=object)

Some annotators marked images as completed in the status sheet but didn't update their annotations layer. I can know which annotators didn't update their layers by comparing the version history of the excel sheet and that of their annotation layer. I will only select the images of annotators that uploaded up to date annotations layer. They are Fiona, Ummamah, Ye, Brian, and Biz.

In [ ]:
# Filter completed annotations
selected_annotators = ['Biz Yoder', 'Brian Mulu Mutua', 'Fiona Bolte-Bradhurst', 'Ummamah Shah', 'Ye Khaung Oo']
filtered_completed_df = completed[completed['Annotator'].isin(selected_annotators)]

# Map annotator names to keys
key_mapping = {
    'Biz Yoder': 'biz',
    'Brian Mulu Mutua': 'mutua',
    'Fiona Bolte-Bradhurst': 'fiona',
    'Ummamah Shah': 'shah',
    'Ye Khaung Oo': 'ye'
}

annotator_image_ids = filtered_completed_df.groupby('Annotator')['Image ID'].apply(list).to_dict()
annotator_image_ids = {key_mapping[old_key]: value for old_key, value in annotator_image_ids.items()}

c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


It looks like there are a lot of polygons not associated with any image. After a lot of investigation, I think these are the potential causes for this high number:
- I found that for some annotators, their polygons don't exist in the images they marked as completed. I verified this carefully for Ye. I opened the images he marked as completed and mapped some polygons for which image_name is null in annotations dataframe. so, some annotators may have not marked images as completed even though they annotated them and updated their annotation layer -> **I later realized that soome missing annotations are under the name of Biz. This is because I kept the 2020 annotations in her dataframe.**
- The annotators used others' annotation layer and added their own annotations to the same layer. 
- Some problem with the CRS of the layers during annotation
- Those polygons come from the annotations of 2020 ECW file the team worked on during the summer. Although I tried to remove the 2020 annotations from the new shapefiles (by selecting path!=previous annotations layer in the function clean_annotation_layers), it seems that those polygons still exist. 

I noticed that some annotators marked some polygons as PV heaters when they clearly aren't. They also marked them with uncertain flag but they are solar panels. That's why, I'm keeping the uncertain and heater marked solar panels in my geodataframe. In the future, we need to process and analyze these polygons further. 

In [ ]:
# Extract the CRS of the images. This will be used to reproject the annotations layers to this CRS to avoid any inconsistencies.
input_tif = r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass\2023_RGB_8cm_W18B_5.tif"
with rasterio.open(input_tif) as dataset:
    geotif_crs = dataset.crs


def clean_annotation_layers(layer):
    copy = layer
    copy = copy.to_crs(geotif_crs)
    copy['area'] = copy.geometry.area
    
    columns_to_drop = ['layer', 'path', 'PV_heater', 'uncertflag', 'PV_Pool', 'PV_pool']
    copy = copy.drop([col for col in columns_to_drop if col in copy.columns], axis=1) 
    
    copy.reset_index(drop=True, inplace=True)       
    
    return copy

In [ ]:
# Define file paths
file_paths = {
    'fiona': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Fiona Bolte-Bradhurst\11.21\bolte_bradhurst_annotations_112124.shp",
    'shah': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Ummamah Shah\Shah_2024.shp",
    'mutua': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Brian Mulu Mutua\mutua_annotations 1027.shp",
    'ye': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Ye Khaung Oo\Ye_annotations.shp",
    'biz': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Biz Yoder\yoder_annotations 1025.shp"
}

# Load and process annotations
annotations_list = []
for annotator, path in file_paths.items():
    annotations = gpd.read_file(path)
    annotations['annotator'] = annotator
    if annotator != 'biz':
        annotations = annotations[annotations['path'].isnull()]
    annotations = clean_annotation_layers(annotations)
    annotations_list.append(annotations)

# Concatenate all annotations and add important polygon features
annotations = pd.concat(annotations_list, ignore_index=True)
annotations.reset_index(drop=True, inplace=True)
annotations['id'] = annotations.index
annotations['centroid'] = annotations.geometry.centroid
annotations['centroid_latitude'] = annotations.centroid.y
annotations['centroid_longitude'] = annotations.centroid.x
annotations.drop(columns=['centroid'], inplace=True)
annotations = annotations[annotations['geometry'].notnull()]

# Add additional columns to annotations
annotations[['image_name', 'nw_corner_of_image_latitude', 'nw_corner_of_image_longitude', 
             'se_corner_of_image_latitude', 'se_corner_of_image_longitude']] = None

annotations

,id,geometry,annotator,area,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,"POLYGON ((-55245.106 -3753790.252, -55222.923 ...",fiona,44.764537,-3.753790e+06,-55233.927554,None,None,None,None,None
1,1,"POLYGON ((-55244.845 -3753792.427, -55243.801 ...",fiona,8.695597,-3.753796e+06,-55243.860816,None,None,None,None,None
2,2,"POLYGON ((-55243.714 -3753800.692, -55222.227 ...",fiona,45.388895,-3.753801e+06,-55233.151068,None,None,None,None,None
3,3,"POLYGON ((-55223.967 -3753790.6, -55222.923 -3...",fiona,8.498829,-3.753794e+06,-55223.009514,None,None,None,None,None
4,4,"POLYGON ((-57165.854 -3758134.693, -57160.226 ...",fiona,30.248899,-3.758133e+06,-57161.713744,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
16223,16223,"POLYGON ((-17705.487 -3769231.7, -17701.322 -3...",biz,4.267862,-3.769231e+06,-17703.207145,None,None,None,None,None
16224,16224,"POLYGON ((-17704.553 -3769232.732, -17701.419 ...",biz,6.051819,-3.769233e+06,-17702.679721,None,None,None,None,None
16225,16225,"POLYGON ((-17602.028 -3769046.931, -17599.44 -...",biz,5.033903,-3.769048e+06,-17601.247101,None,None,None,None,None
16226,16226,"POLYGON ((-17597.922 -3769049.169, -17591.772 ...",biz,11.997585,-3.769052e+06,-17595.268258,None,None,None,None,None


Next, we need to determine the image each polygon belongs to. We are doing this by calculating the border coordinates for each annotator's images, and then checking which polygon is within the bounds of the image.

In [ ]:
def get_image_border_coordinates(image_path):
    with rasterio.open(image_path) as src:
        return src.bounds

folder_path = r'C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass'
prefix = '2023_RGB_8cm_'

# Dictionary to store border coordinates for each annotator's images
annotator_border_coordinates = {}
completed_ann = []

# Dictionary to store already processed images
processed_images = {}

# Iterate over each annotator and add their image names
for annotator, image_names in annotator_image_ids.items():
    border_coordinates = {}
    for image_name in image_names:
        full_name = f"{prefix}{image_name}.tif"
        image_path = os.path.join(folder_path, full_name)
        
        if os.path.exists(image_path):
            if image_name not in processed_images:
                coordinates = get_image_border_coordinates(image_path)
                processed_images[image_name] = coordinates
            else:
                coordinates = processed_images[image_name]
            
            border_coordinates[image_name] = coordinates
    
    annotator_border_coordinates[annotator] = border_coordinates
    completed_ann.append(annotator)

In [ ]:
BoundingBox = namedtuple('BoundingBox', ['left', 'bottom', 'right', 'top'])

transformed_dict = {}
for annotator, images in annotator_border_coordinates.items():
    transformed_dict[annotator] = {image: {'left': bounds.left, 'bottom': bounds.bottom, 'right': bounds.right, 'top': bounds.top} for image, bounds in images.items()}

annotator_border_coordinates = transformed_dict 

In [ ]:
# Function to check if the centroid (consequently the polygon) is within a bounding box
def is_point_within_bounds(lat, lon, bounds):
    return bounds['left'] <= lon <= bounds['right'] and bounds['bottom'] <= lat <= bounds['top']

In [ ]:
flattened_coordinates = {}
for annotator, images in annotator_border_coordinates.items():
    flattened_coordinates.update(images)

# Iterate over each row in the annotations DataFrame
for idx, row in annotations.iterrows():
    # annotator = row['annotator']
    centroid_lat = row['centroid_latitude']
    centroid_lon = row['centroid_longitude']
    
    # Check which image the centroid belongs to
    for image_name, bounds in flattened_coordinates.items():
        if is_point_within_bounds(centroid_lat, centroid_lon, bounds):
            annotations.loc[idx, 'image_name'] = image_name
            annotations.loc[idx, 'nw_corner_of_image_latitude'] = bounds['top']
            annotations.loc[idx, 'nw_corner_of_image_longitude'] = bounds['left']
            annotations.loc[idx, 'se_corner_of_image_latitude'] = bounds['bottom']
            annotations.loc[idx, 'se_corner_of_image_longitude'] = bounds['right']
            break

# annotations

,id,geometry,annotator,area,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,"POLYGON ((-55245.106 -3753790.252, -55222.923 ...",fiona,44.764537,-3.753790e+06,-55233.927554,None,None,None,None,None
1,1,"POLYGON ((-55244.845 -3753792.427, -55243.801 ...",fiona,8.695597,-3.753796e+06,-55243.860816,None,None,None,None,None
2,2,"POLYGON ((-55243.714 -3753800.692, -55222.227 ...",fiona,45.388895,-3.753801e+06,-55233.151068,None,None,None,None,None
3,3,"POLYGON ((-55223.967 -3753790.6, -55222.923 -3...",fiona,8.498829,-3.753794e+06,-55223.009514,None,None,None,None,None
4,4,"POLYGON ((-57165.854 -3758134.693, -57160.226 ...",fiona,30.248899,-3.758133e+06,-57161.713744,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
16223,16223,"POLYGON ((-17705.487 -3769231.7, -17701.322 -3...",biz,4.267862,-3.769231e+06,-17703.207145,None,None,None,None,None
16224,16224,"POLYGON ((-17704.553 -3769232.732, -17701.419 ...",biz,6.051819,-3.769233e+06,-17702.679721,None,None,None,None,None
16225,16225,"POLYGON ((-17602.028 -3769046.931, -17599.44 -...",biz,5.033903,-3.769048e+06,-17601.247101,None,None,None,None,None
16226,16226,"POLYGON ((-17597.922 -3769049.169, -17591.772 ...",biz,11.997585,-3.769052e+06,-17595.268258,None,None,None,None,None


In [ ]:
# available_annotations = annotations[annotations['image_name'].notnull()]
# available_annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\available_annotations.shp")

C:\Users\AICHA\AppData\Local\Temp\ipykernel_31352\1511238504.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  available_annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\available_annotations.shp")
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'centroid_latitude' to 'centroid_l'
  ogr_write(
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'centroid_longitude' to 'centroid_1'
  ogr_write(
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'nw_corner_of_image_latitude' to 'nw_corner_'
  ogr_write(
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: Runt

Now, we calculate the pixel coordinates and the area in pixels of each polygon.

In [112]:
annotations = annotations[annotations['image_name'].notnull()]

def calculate_pixel_metrics_grouped(group, image_base_path):
    metrics = []
    for _, row in group.iterrows():
        geotiff_path = image_base_path + row['image_name'] + ".tif"
        try:
            with rasterio.open(geotiff_path) as src:
                transform = src.transform
                metric = calculate_pixel_metrics_dynamic(row, transform)
                metrics.append(metric)
        except FileNotFoundError:
            print(f"GeoTIFF not found: {geotiff_path}")
            metrics.append(None)
    
    return pd.DataFrame(metrics, index=group.index)

def calculate_pixel_metrics_dynamic(row, transform):
    polygon = shape(row['geometry'])
    centroid = polygon.centroid

    centroid_pixel_row, centroid_pixel_col = rowcol(transform, centroid.x, centroid.y)

    pixel_vertices = [
        rowcol(transform, vertex[0], vertex[1]) for vertex in polygon.exterior.coords
    ]

    pixel_polygon = shape({
        'type': 'Polygon',
        'coordinates': [[(c, r) for r, c in pixel_vertices]]
    })
    pixel_area = pixel_polygon.area

    return {
        'polygon_vertices_pixels': pixel_vertices,
        'centroid_latitude_pixels': centroid_pixel_row,
        'centroid_longitude_pixels': centroid_pixel_col,
        'area_pixels': pixel_area
    }

# Group the annotations by image_name
grouped_annotations = annotations.groupby('image_name')


image_base_path = r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass\2023_RGB_8cm_"
pixel_metrics_list = []

for name, group in tqdm(grouped_annotations, desc="Processing Images"):
    print(f"Processing image: {name}")
    metrics = calculate_pixel_metrics_grouped(group, image_base_path)
    pixel_metrics_list.append(metrics)

# Concatenate all metrics into a single DataFrame
pixel_metrics_df = pd.concat(pixel_metrics_list)

# Assign each metric to its respective column in the annotations DataFrame
annotations = annotations.join(pixel_metrics_df)

Processing Images:   0%|          | 0/113 [00:00<?, ?it/s]

Processing image: W07C_10


Processing Images:   1%|          | 1/113 [00:03<05:49,  3.12s/it]

Processing image: W07C_11


Processing Images:   2%|▏         | 2/113 [00:10<10:45,  5.81s/it]

...

Processing Images:  98%|█████████▊| 111/113 [49:05<01:09, 34.78s/it]

Processing image: W57B_9


Processing Images:  99%|█████████▉| 112/113 [49:31<00:32, 32.21s/it]

Processing image: W57C_9


Processing Images: 100%|██████████| 113/113 [50:06<00:00, 26.60s/it]


In [ ]:
# annotations.columns = ['id', 'annotator', 'area', 'centroid_latitude', 'centroid_longitude',
#        'image_name', 'nw_corner_of_image_latitude',
#        'nw_corner_of_image_longitude', 'se_corner_of_image_latitude',
#        'se_corner_of_image_longitude', 'polygon_vertices_pixels',
#        'centroid_latitude_pixels', 'centroid_longitude_pixels', 'area_pixels',
#        'geometry']

# annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\annotations_final.geojson")

In [ ]:
annotations = gpd.read_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\annotations_final.geojson")
annotations

,id,annotator,area,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude,polygon_vertices_pixels,centroid_latitude_pixels,centroid_longitude_pixels,area_pixels,geometry
0,7,fiona,19.432886,-3.776356e+06,-5578.396188,W07C_10,-3776000.0,-6000.0,-3777000.0,-5000.0,"[(np.int32(4492), np.int32(5226)), (np.int32(4...",4444,5270,2995.5,"POLYGON ((-5581.91047 -3776359.36031, -5576.61..."
1,8,fiona,19.257815,-3.776357e+06,-5575.799560,W07C_10,-3776000.0,-6000.0,-3777000.0,-5000.0,"[(np.int32(4405), np.int32(5323)), (np.int32(4...",4464,5302,2982.0,"POLYGON ((-5574.12577 -3776352.40598, -5572.41..."
2,9,fiona,18.028277,-3.776358e+06,-5573.512491,W07C_10,-3776000.0,-6000.0,-3777000.0,-5000.0,"[(np.int32(4421), np.int32(5353)), (np.int32(4...",4480,5331,2866.0,"POLYGON ((-5571.68657 -3776353.75533, -5570.12..."
3,10,fiona,11.137212,-3.776075e+06,-5752.630109,W07C_10,-3776000.0,-6000.0,-3777000.0,-5000.0,"[(np.int32(926), np.int32(3054)), (np.int32(92...",938,3092,1698.5,"POLYGON ((-5755.66492 -3776074.129, -5749.6447..."
4,11,fiona,32.433931,-3.776077e+06,-5746.771046,W07C_10,-3776000.0,-6000.0,-3777000.0,-5000.0,"[(np.int32(951), np.int32(3054)), (np.int32(95...",962,3165,5029.5,"POLYGON ((-5755.61302 -3776076.10112, -5737.81..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7704,16194,biz,3.208093,-3.769770e+06,-18127.213491,W16C_22,-3769000.0,-19000.0,-3770000.0,-18000.0,"[(np.int32(9614), np.int32(10894)), (np.int32(...",9620,10909,489.5,"POLYGON ((-18128.46558 -3769769.19484, -18127...."
7705,16195,biz,3.105449,-3.769768e+06,-18125.832320,W16C_22,-3769000.0,-19000.0,-3770000.0,-18000.0,"[(np.int32(9599), np.int32(10911)), (np.int32(...",9604,10927,489.5,"POLYGON ((-18127.10326 -3769767.94929, -18126...."
7706,16196,biz,3.100336,-3.769766e+06,-18122.758878,W16C_22,-3769000.0,-19000.0,-3770000.0,-18000.0,"[(np.int32(9563), np.int32(10949)), (np.int32(...",9569,10965,498.0,"POLYGON ((-18124.00884 -3769765.08841, -18123...."
7707,16197,biz,3.274565,-3.769764e+06,-18121.248834,W16C_22,-3769000.0,-19000.0,-3770000.0,-18000.0,"[(np.int32(9546), np.int32(10968)), (np.int32(...",9552,10984,501.0,"POLYGON ((-18122.54921 -3769763.68717, -18121...."


Some necessary data manipulation of the annotations dataframe's polygon_vertices_pixels column before creating image mask pairs.

In [ ]:
df = annotations.copy()

# Clean the strings to remove np.int32 calls and fix any incomplete strings
df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(
    lambda x: re.sub(r'np\.int32\((\d+)\)', r'\1', x)
)

# Replace parentheses with square brackets
df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(
    lambda x: re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', x)
)

# Ensure all tuples are properly closed
def fix_incomplete_tuples(polygon_str):
    # Find all tuples
    tuples = re.findall(r'\[\d+, \d+\]', polygon_str)
    # Join them back into a string
    fixed_str = '[' + ', '.join(tuples) + ']'
    return fixed_str

df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(fix_incomplete_tuples)

# Convert the string representation of lists back to actual lists
df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(
    lambda x: ast.literal_eval(x)
)

print(df['polygon_vertices_pixels'])

0       [[4492, 5226], [4385, 5292], [4397, 5313], [45...
1       [[4405, 5323], [4416, 5344], [4524, 5281], [45...
2       [[4421, 5353], [4432, 5373], [4539, 5309], [45...
3       [[926, 3054], [927, 3129], [949, 3130], [949, ...
4       [[951, 3054], [950, 3277], [972, 3278], [974, ...
                              ...                        
7704    [[9614, 10894], [9605, 10905], [9623, 10926], ...
7705    [[9599, 10911], [9588, 10923], [9608, 10943], ...
7706    [[9563, 10949], [9553, 10961], [9573, 10982], ...
7707    [[9546, 10968], [9536, 10979], [9555, 11001], ...
7708    [[9444, 11072], [9431, 11086], [9450, 11109], ...
Name: polygon_vertices_pixels, Length: 7709, dtype: object


In [ ]:
annotations = df

Now that the dataframe is ready, we can finally create the image mask pairs. Each image/tile is 896x896 pixels

In [103]:
def create_mask(image_shape, polygons):
    mask = np.zeros(image_shape[:2], dtype="uint8")
    for polygon in polygons:
        cv2.fillPoly(mask, [polygon], 255)
    flipped_mask = cv2.flip(mask, 0)
    rotated_mask = cv2.rotate(flipped_mask, cv2.ROTATE_90_CLOCKWISE)
    return rotated_mask

def save_tile_and_mask(tile, mask, tile_index_pixels, tile_dir, mask_dir, image_name):
    tile_filename = os.path.join(tile_dir, f'i_{image_name}_{tile_index_pixels}.png')
    mask_filename = os.path.join(mask_dir, f'm_{image_name}_{tile_index_pixels}.png')
    cv2.imwrite(tile_filename, cv2.cvtColor(tile, cv2.COLOR_RGB2BGR))
    cv2.imwrite(mask_filename, mask)

def adjust_polygon_coordinates(polygons, x_offset, y_offset):
    adjusted_polygons = []
    for polygon in polygons:
        adjusted_polygon = polygon - np.array([x_offset, y_offset])
        adjusted_polygons.append(adjusted_polygon)
    return adjusted_polygons

def process_geotiff(image_name, geotiff_path, tile_size, df, tile_dir, mask_dir):
    with rasterio.open(geotiff_path) as src:
        geotiff_array = src.read()

        if len(geotiff_array.shape) == 3:
            geotiff_array = np.transpose(geotiff_array, (1, 2, 0))

        height, width = geotiff_array.shape[:2]

        # Calculate padding
        pad_height = (tile_size - height % tile_size) % tile_size
        pad_width = (tile_size - width % tile_size) % tile_size

        # Add padding to the image
        padded_image = np.pad(geotiff_array, ((0, pad_height), (0, pad_width), (0, 0)), mode='constant')

        padded_height, padded_width = padded_image.shape[:2]

        tile_index = 0
        for y in range(0, padded_height, tile_size):
            for x in range(0, padded_width, tile_size):
                tile = padded_image[y:y+tile_size, x:x+tile_size]
                # print('tile pixel edges: ', y, y+tile_size, x, x+tile_size)

                polygons_in_tile = []
                for _, row in df.iterrows():
                    # polygon_str = row['polygon_vertices_pixels']
                    # polygon = np.array(ast.literal_eval(polygon_str), dtype=np.int32)
                    polygon = row['polygon_vertices_pixels']
                    
                    bounds = {
                        'left': x,
                        'right': x+tile_size,
                        'bottom': y+tile_size,
                        'top': y
                    }
                    # print(bounds)
                    
                    if (bounds['left'] <= row['centroid_longitude_pixels'] <= bounds['right'] and bounds['top'] <= row['centroid_latitude_pixels'] <= bounds['bottom']):
                        polygons_in_tile.append(polygon)
                        # print(row['id'])
                    
                # print(polygons_in_tile)
                # print(x)
                # print(y)
                adjusted_polygons = adjust_polygon_coordinates(polygons_in_tile, y, x)
                # print(adjusted_polygons)

                mask = create_mask(tile.shape, adjusted_polygons)

                tile_index_pixels = str(int(y/tile_size)) + "_" + str(int(x/tile_size))
                save_tile_and_mask(tile, mask, tile_index_pixels, tile_dir, mask_dir, image_name)
                tile_index += 1


def process_all_images_in_folder(folder_path, annotations_df, tile_size, tile_dir, mask_dir):
    unique_images = annotations_df['image_name'].unique()
    
    for image_name in unique_images:
        image_path = folder_path + image_name + ".tif"
        
        if os.path.exists(image_path):
            print(image_name)
            image_annotations_df = annotations_df[annotations_df['image_name'] == image_name]
            
            process_geotiff(image_name, image_path, tile_size, image_annotations_df, tile_dir, mask_dir)


tile_dir = r'C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\dataset\tiles'
mask_dir = r'C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\dataset\masks'
os.makedirs(tile_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

dataset_path = r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass\2023_RGB_8cm_"
tile_size = 896 

process_all_images_in_folder(dataset_path, annotations, tile_size=896, tile_dir=tile_dir, mask_dir=mask_dir)


W07C_10
W07C_11
W07C_12
W07C_13
W07C_16
W57A_17
W57A_18
W57A_19
W57A_21
W57A_22
W57A_23
W57A_24
W57A_25
W57B_2
W57B_3
W57B_4
W57B_6
W57B_7
W57B_8
W57B_9
W57B_11
W57B_12
W57B_18
W57B_13
W57B_10
W57B_19
W57B_20
W57C_9
W48C_6
W48C_11
W48C_16
W48C_22
W49A_11
W49A_2
W49A_16
W50D_4
W50D_5
W50D_25
W33A_9
W33A_10
W33A_18
W33B_2
W33B_16
W33B_19
W33C_7
W07C_17
W07C_2
W07C_21
W07C_22
W07C_23
W07C_3
W07C_4
W07C_5
W07C_6
W07C_7
W07C_8
W07C_9
W07D_1
W07D_6
W08A_1
W08A_12
W08A_2
W08B_4
W08B_9
W12A_17
W12A_18
W12A_21
W12A_22
W12A_23
W12A_24
W12C_11
W12C_12
W12C_16
W12C_17
W12C_3
W12C_4
W12C_6
W13A_1
W13C_11
W13C_13
W13C_16
W13C_17
W13C_18
W13C_21
W13C_22
W13C_8
W14A_11
W14A_12
W14A_16
W14A_21
W16C_14
W16C_15
W16C_17
W16C_18
W16C_19
W18B_5
W18B_9
W18B_8
W16D_17
W57B_5
W57B_14
W16C_22
W16D_25
W16D_24
W16C_20
W16C_21
W16D_11
W16D_12
W16D_16
W16D_21
W16D_22
W16D_23
W16D_7


There are a lot of images that don't contain any solar panel. To avoid wasting training resources, we'll only select the images that contain panels to work with.

In [ ]:
# copy the masks that contain the target to a seperate folder

def check_mask_has_target(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    return np.any(mask > 0)

def copy_masks_with_target(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)
        
        if check_mask_has_target(file_path):
            shutil.copy(file_path, destination_folder)
            print(f"Copied {filename} to {destination_folder}")

source_folder = '/home/as1233/data/cape_town/masks'
destination_folder = '/home/as1233/data/cape_town/masks_target'

copy_masks_with_target(source_folder, destination_folder)

In [ ]:
# copy the corresponding images to a new folder
def copy_corresponding_images(mask_folder, image_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for mask_filename in os.listdir(mask_folder):
        image_filename = 'i' + mask_filename[1:]
        image_path = os.path.join(image_folder, image_filename)
        
        if os.path.exists(image_path):
            shutil.copy(image_path, destination_folder)
            print(f"Copied {image_filename} to {destination_folder}")

mask_folder = '/home/as1233/data/cape_town/masks_target'
image_folder = '/home/as1233/data/cape_town/tiles'
destination_folder = '/home/as1233/data/cape_town/images_target'

copy_corresponding_images(mask_folder, image_folder, destination_folder)

In [ ]:
# split the dataset into train, test, and val datasets

def split_data(mask_folder, image_folder, train_folder, val_folder, test_folder, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'masks'), exist_ok=True)
    
    mask_files = os.listdir(mask_folder)
    
    random.shuffle(mask_files)
    
    total_files = len(mask_files)
    train_count = int(total_files * train_ratio)
    val_count = int(total_files * val_ratio)
    test_count = total_files - train_count - val_count
    
    train_files = mask_files[:train_count]
    val_files = mask_files[train_count:train_count + val_count]
    test_files = mask_files[train_count + val_count:]
    
    def copy_files(file_list, dest_image_folder, dest_mask_folder):
        for mask_filename in file_list:
            shutil.copy(os.path.join(mask_folder, mask_filename), dest_mask_folder)
            
            image_filename = 'i' + mask_filename[1:]
            shutil.copy(os.path.join(image_folder, image_filename), dest_image_folder)
    
    copy_files(train_files, os.path.join(train_folder, 'images'), os.path.join(train_folder, 'masks'))
    copy_files(val_files, os.path.join(val_folder, 'images'), os.path.join(val_folder, 'masks'))
    copy_files(test_files, os.path.join(test_folder, 'images'), os.path.join(test_folder, 'masks'))

mask_folder = '/home/as1233/data/cape_town/masks_target'
image_folder = '/home/as1233/data/cape_town/images_target'
train_folder = '/home/as1233/data/cape_town/train'
val_folder = '/home/as1233/data/cape_town/val'
test_folder = '/home/as1233/data/cape_town/test'

split_data(mask_folder, image_folder, train_folder, val_folder, test_folder)